Start with Shrinkage models:

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LassoLarsCV
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_absolute_error


In [ ]:
data = pd.read_csv('ML_price_data.csv', index_col=0, parse_dates=True)
data = data.head()

In [ ]:
data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
